In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import panel as pn

pn.extension()

# Carregar o dataset TRATADO (criado no relatório)
df = pd.read_csv("computadores_tratado.csv")

# Se por algum motivo ainda não tiveres estas colunas no CSV,
# este bloco volta a criá-las (se já existirem, apenas sobrescreve, não faz mal).
from datetime import datetime
ano_ref = datetime.now().year

if "has_battery" not in df.columns:
    df["has_battery"] = df["battery_wh"] > 0

if "age_years" not in df.columns:
    df["age_years"] = ano_ref - df["release_year"]

if "is_high_ram" not in df.columns:
    df["is_high_ram"] = df["ram_gb"] >= 16

if "is_gaming" not in df.columns:
    df["is_gaming"] = (df["vram_gb"] >= 8) & (df["is_high_ram"]) & (df["refresh_hz"] >= 120)

if "is_bundle" not in df.columns:
    df["is_bundle"] = df["device_type"] == "Desktop"

if "performance_score" not in df.columns:
    df["performance_score"] = df["gpu_tier"] * 2 + df["cpu_tier"]

# Garantir faixas de preço
bins_price = [0, 500, 1000, 2000, df["price"].max()]
labels_price = ["até 500", "501–1000", "1001–2000", "mais de 2000"]
df["price_band"] = pd.cut(df["price"], bins=bins_price, labels=labels_price)

# Garantir faixas de idade
bins_age = [-1, 1, 3, 5, 100]
labels_age = ["até 1 ano", "2–3 anos", "4–5 anos", "mais de 5 anos"]
df["age_band"] = pd.cut(df["age_years"], bins=bins_age, labels=labels_age)

df.head()


In [ ]:
# Opções para os filtros
device_type_options = ["Todos"] + sorted(df["device_type"].dropna().unique().tolist())
gaming_options = ["Todos", "Gaming", "Não gaming"]
price_band_options = ["Todas"] + [str(x) for x in df["price_band"].cat.categories]

# Widgets de filtro
w_device = pn.widgets.Select(name="Tipo de equipamento", options=device_type_options, value="Todos")
w_gaming = pn.widgets.Select(name="Perfil gaming", options=gaming_options, value="Todos")
w_price_band = pn.widgets.Select(name="Faixa de preço", options=price_band_options, value="Todas")

w_year = pn.widgets.IntRangeSlider(
    name="Ano de lançamento",
    start=int(df["release_year"].min()),
    end=int(df["release_year"].max()),
    value=(int(df["release_year"].min()), int(df["release_year"].max()))
)

# Widget para escolher o tipo de gráfico principal
w_plot = pn.widgets.Select(
    name="Tipo de gráfico",
    options=[
        "bar_preco_tipo",
        "box_preco_gaming",
        "scatter_perf_preco",
    ],
    value="bar_preco_tipo",
)


In [ ]:
def filtrar_df(device_type, gaming, price_band, year_range):
    df_filtrado = df.copy()
    
    # filtro device_type
    if device_type != "Todos":
        df_filtrado = df_filtrado[df_filtrado["device_type"] == device_type]
    
    # filtro gaming
    if gaming == "Gaming":
        df_filtrado = df_filtrado[df_filtrado["is_gaming"] == True]
    elif gaming == "Não gaming":
        df_filtrado = df_filtrado[df_filtrado["is_gaming"] == False]
    
    # filtro price_band
    if price_band != "Todas":
        df_filtrado = df_filtrado[df_filtrado["price_band"].astype(str) == price_band]
    
    # filtro por ano
    year_min, year_max = year_range
    df_filtrado = df_filtrado[
        (df_filtrado["release_year"] >= year_min) &
        (df_filtrado["release_year"] <= year_max)
    ]
    
    return df_filtrado


In [ ]:
def grafico_principal(df_filtrado, tipo_grafico):
    fig, ax = plt.subplots()

    # caso sem dados
    if df_filtrado.empty:
        ax.text(0.5, 0.5, "Sem dados para os filtros selecionados", ha="center", va="center")
        ax.axis("off")
        return fig

    # 1) Gráfico de barras – preço médio por tipo de equipamento
    if tipo_grafico == "bar_preco_tipo":
        serie = df_filtrado.groupby("device_type")["price"].mean()
        serie.plot(kind="bar", ax=ax)
        ax.set_ylabel("Preço médio")
        ax.set_title("Preço médio por tipo de equipamento")

    # 2) Boxplot – preço por perfil gaming
    elif tipo_grafico == "box_preco_gaming":
        df_filtrado.boxplot(column="price", by="is_gaming", ax=ax)
        ax.set_ylabel("Preço")
        ax.set_title("Preço por perfil gaming (gaming vs não gaming)")
        plt.suptitle("")  # tira o título extra feio

    # 3) Scatter – desempenho vs preço
    elif tipo_grafico == "scatter_perf_preco":
        ax.scatter(df_filtrado["performance_score"], df_filtrado["price"], alpha=0.5)
        ax.set_xlabel("Índice de desempenho (performance_score)")
        ax.set_ylabel("Preço")
        ax.set_title("Relação entre desempenho e preço")

    return fig


In [ ]:
def tabela_resumo(df_filtrado):
    if df_filtrado.empty:
        return "Sem dados para os filtros selecionados."
    
    resumo = df_filtrado[["price", "performance_score", "ram_gb", "age_years"]].describe().round(2)
    return resumo

def grafico_temporal(df_filtrado):
    fig, ax = plt.subplots()

    if df_filtrado.empty:
        ax.text(0.5, 0.5, "Sem dados para os filtros selecionados", ha="center", va="center")
        ax.axis("off")
        return fig

    serie = df_filtrado.groupby("release_year")["price"].mean().sort_index()
    serie.plot(kind="line", marker="o", ax=ax)
    ax.set_xlabel("Ano de lançamento")
    ax.set_ylabel("Preço médio")
    ax.set_title("Preço médio por ano (dados filtrados)")
    return fig


In [ ]:
# df filtrado dinamicamente pelos widgets
filtro_bind = pn.bind(
    filtrar_df,
    device_type=w_device,
    gaming=w_gaming,
    price_band=w_price_band,
    year_range=w_year,
)

# gráfico principal dinâmico
grafico_bind = pn.bind(
    grafico_principal,
    df_filtrado=filtro_bind,
    tipo_grafico=w_plot
)

# gráfico temporal dinâmico
grafico_temp_bind = pn.bind(
    grafico_temporal,
    df_filtrado=filtro_bind
)

# tabela resumo dinâmica
tabela_bind = pn.bind(
    tabela_resumo,
    df_filtrado=filtro_bind
)


In [ ]:
# Aba 1 – Gráfico principal
dash_graficos = pn.Column(
    "### Gráfico principal",
    w_plot,
    grafico_bind,
)

# Aba 2 – Séries temporais
dash_temporal = pn.Column(
    "### Séries temporais (preço médio por ano)",
    grafico_temp_bind,
)

# Aba 3 – Tabela resumo
dash_resumo = pn.Column(
    "### Resumo estatístico dos dados filtrados",
    tabela_bind,
)

# Tabs principais (como nos exemplos do stor)
main = pn.Tabs(
    ("Gráficos", dash_graficos),
    ("Séries temporais", dash_temporal),
    ("Resumo estatístico", dash_resumo),
)

# Template Material, ao estilo dos exemplos
app = pn.template.MaterialTemplate(
    header_background='red',
    title='Dashboard – Computadores',
    main=[main],
    sidebar=[w_device, w_gaming, w_price_band, w_year],
)

app.servable()
